# Function

In [ ]:
def create_points_along_line(line, interval):
    length = line.length
    current_distance = 0
    points = []

    while current_distance < length:
        point = line.interpolate(current_distance)
        points.append(Point(point.x, point.y))
        current_distance += decimal_degrees

    return points

# Create MultiPoint along line.shp

In [ ]:
import math
from shapely.geometry import Point
import pystac_client
import planetary_computer
import geopandas as gpd

# Path to your shapefile
shapefile_path = "C:\\Path\\To\\Desktop\\Shapefile\\TestPODbndry_BUCK_2.shp"

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Create a buffer around each geometry in the GeoDataFrame
buffered_gdf = gdf.buffer(0.01)

# Create a new GeoDataFrame from the buffered geometries
buffered_gdf = gpd.GeoDataFrame(geometry=buffered_gdf, crs=gdf.crs)

# Explore the buffered GeoDataFrame
buffered_gdf.explore()

meters_between_PTS=1000
xmin, ymin, xmax, ymax = buffered_gdf.total_bounds
latitude = (ymin + ymax) / 2
decimal_degrees = -meters_between_PTS / (111319.9 * math.cos(latitude))
print(decimal_degrees)

# Apply the function to each line in breached_pod and collect all points
all_points = []
for line in gdf.geometry:
    points = create_points_along_line(line, decimal_degrees)
    all_points.extend(points)

# Convert list of points to a GeoDataFrame
points_gdf = gpd.GeoDataFrame(geometry=all_points, crs=gdf.crs)

coordinates_list = [(point.x, point.y) for point in points_gdf.geometry]

# Construct the GeoJSON MultiPoint object
loc = {
    "type": "MultiPoint",
    "coordinates": coordinates_list
}


# Plot pts

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Assuming you have created the 'points_gdf' GeoDataFrame with the points

# Plot the buffered geometries from your original GeoDataFrame
ax = buffered_gdf.plot(figsize=(10, 10), color='lightblue', edgecolor='black')

# Plot the points on top of the buffered geometries
points_gdf.plot(ax=ax, color='red', markersize=5)

# Set plot title and labels
ax.set_title("Buffered Geometries with Points")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# Show the plot
plt.show()


# DOWNLOAD multiple tile from multiple collection along the multipoint (loc)

In [ ]:
from shapely.geometry import shape
from shapely.ops import transform

geom = shape(loc)
geom_buff=gpd.GeoSeries(geom.buffer(0.01),crs='EPSG:4326')
#geom_buff.explore()

#time_of_interest = "2013-01-01/2020-12-31"

# activate catelog and search for dataset
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=planetary_computer.sign_inplace)

# Collections for LIDAR
COLLECTIONLIST = [
"3dep-lidar-hag",
"3dep-lidar-dsm",
#"3dep-lidar-pointsourceid",
#"3dep-lidar-intensity",
"3dep-lidar-dtm",
#"3dep-lidar-dtm-native",
#"3dep-lidar-returns",
#"3dep-lidar-classification"
                 ]
for w in range(len(COLLECTIONLIST)):
    
    srch_hag = catalog.search(collections=COLLECTIONLIST[w], intersects=geom.buffer(0.01))#, datetime=time_of_interest)

    ic_hag = srch_hag.get_all_items()
    df_hag = gpd.GeoDataFrame.from_features(ic_hag.to_dict(), crs="epsg:4326")
    #display(df_hag.head(5))
    #p1=geom_buff.explore(color='orange')
    #p2=df_hag.explore(m=p1,color='blue')
    #p2

    # Download data
    #ic_hag[0].assets['data'].href

    Number_of_Tiles = len(ic_hag)

    import requests

    for i in range(Number_of_Tiles):
        print(i)
        # Select the STAC item you want to export
        item = ic_hag[i]  # Replace 0 with the index of the item you want to export

        # Get the download URL of the 'data' asset
        asset_url = item.assets['data'].href

        # Define the output GeoTIFF file path
        output_tif_path = f"C:\\Path\\To\\Desktop\\ExportFolder\\{COLLECTIONLIST[w]}_{i}.tif"

        # Download the asset
        response = requests.get(asset_url)

        # Check if the download was successful
        if response.status_code == 200:
            with open(output_tif_path, 'wb') as output_file:
                output_file.write(response.content)
            print(f"GeoTIFF file '{output_tif_path}' has been downloaded.")
        else:
            print(f"Failed to download the asset.")